**<h1>Zdjęcia Aut i Kotów</h1>**

<h1>Wstęp</h1>
Pamiętasz Marka Adamczyka, który prosił cię o pomoc w algorytmie znajdującym twórcę tweeta? Okazało się że ten pomysł to był strzał w dziesiątkę! Niestety przed wyjechaniem w podróż dookoła świata zapomniał ci on dać twoich należności za wykonaną pracę. Gdy po wielu miesiącach udało ci się z nim skontaktować, wspomniał ci on, że wczoraj zauważył, że zdjęcia samochodów, kotów, oraz innych fascynujących rzeczy, które zrobił są na tyle niewyraźne, że algorytm w jego telefonie nie jest w stanie ich automatycznie przypisać, i że przez będzie zmuszony do potwórzenia tej podróży. Niestety nie jest on ich w stanie sam przyporządkować ponieważ podczas tej podróży udało mu się zrobić tych zdjęć około... 60000. Nie masz pojęcia, jakim sposobem był on w stanie zrobić tyle zdjęć, lecz wiesz, że jeśli wybierze się w kolejną podróż, to raczej już nie zobaczysz swoich pieniędzy. Postanowiłeś więc zaproponować mu, iż napiszesz program, który automatycznie przyporządkuje zdjęcia do klas, których automat w jego telefonie nie był w stanie rozwiązać. Wywiąż się z obietnicy, jeśli wynik będzie zadowalający, to być może zobaczysz twoje należne pieniądze

# <h2>Dostarczone pliki</h2>

- `train_labeled.csv` - Dane treningowe z etykietami
- `train_unlabeled.csv` - Dane treningowe bez etykiet
- `test.csv` - Dane testowe
- `przykodp.csv` - Przykładowy plik w formacie w jakim ma być `odpowiedzi.csv`.
- `Zadanie.ipynb` - Notebook startowy do pracy nad modelem.

# <h2>Foldery z obrazami</h2>

  -  `/train_labeled` - Zawiera obrazy odpowiadające etykietom z train_labeled.csv.
  -  `/train_unlabeled` - Zawiera obrazy odpowiadające danym z train_unlabeled.csv (bez etykiet).
  -  `/test` - Zawiera obrazy do testowania.
# <h2>Twoje zadanie</h2>

### **1. Self-supervised Pretraining**
- Zbuduj własną architekturę modelu.
- Naucz model reprezentacji na danych `train_unlabeled.csv` za pomocą self-supervised learning. Możesz użyć metod takich jak:
  - Contrastive learning (np. SimCLR).
  - Autoenkodery.

### **2. Fine-tuning**
- Użyj wytrenowanego modelu do klasyfikacji na danych z częściowymi etykietami (`train_labeled.csv`).
- Dostosuj hiperparametry (np. liczba epok, batch size, learning rate) dla swojego modelu.

### **3. Ewaluacja**
- Przeprowadź ewaluację swojego modelu na zbiorze testowym (`test.csv`).

# <h2>Ograniczenia</h2>

- Czas działania twojego kodu(trening i ewaluacja) na T4 na Google Colab powinien wynosić maksymalnie 10 minut.
- Nie wolno korzystać z uprzednio wytrenowanych modeli oraz ze zbiorów danych innych niż dostarczony.
- Twój kod może trenować się tylko i wyłącznie na danych z pliku train.csv.
- Wszystkie dopuszczalne biblioteki są dostępne w pliku requirements.txt

# <h2>Ocenanie</h2>

Ocena zależy od wartości accuracy w sposób liniowy, przy czym:

- Jeśli `accuracy < 0.5`, to liczba punktów wynosi 0.
- Jeśli `accuracy >= 0.7`, to liczba punktów wynosi 1.
- Dla wartości `accuracy` pomiędzy 0.5 a 0.7, liczba punktów rośnie liniowo.

Wzór na obliczenie punktów (P):

$$ P = \frac{accuracy - 0.5}{0.2} $$

# <h2>Ograniczenia</h2>

1. Kod może korzystać wyłącznie z danych dostarczonych w plikach `train_labeled.csv`, `train_unlabeled.csv` oraz `test.csv`.
2. Czas treningu i ewaluacji na GPU T4 w Google Colab powinien wynosić maksymalnie 10 minut.
3. Użytkownik musi samodzielnie zbudować model od podstaw, bez korzystania z gotowych modeli (np. ResNet, VGG).

<h2>Rozwiązanie</h2>

- W tym zadaniu musisz musisz dołączyć plik Zadanie.ipynb, który po włączeniu utworzy plik odpowiedzi.csv gdzie będą znajdowały sie odpowiedzi, w formacie takim jak przykodp.csv, oraz plik odpowiedzi.csv.



In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import pandas as pd
from PIL import Image
import os


In [ ]:
class ImageDataset(Dataset):
    def __init__(self, csv_path, transform=None, labeled=True):
        """
        Args:
            csv_path (str): Path to the CSV file.
            transform (callable, optional): A function/transform to apply to the images.
            labeled (bool): Whether the dataset includes labels.
        """
        self.data = pd.read_csv(csv_path)
        self.transform = transform
        self.labeled = labeled

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 0]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        if self.labeled:
            label = self.data.iloc[idx, 1]
            return image, label
        else:
            return image
    
def load_datasets():
    """
    Funkcja ładująca dane z plików CSV i przygotowująca DataLoadery.
    
    TODO:
    1. Wczytaj ścieżki do plików CSV dla zbiorów danych (np. train_labeled.csv, train_unlabeled.csv, test.csv).
    2. Zdefiniuj odpowiednie transformacje dla obrazów (np. augmentacja, normalizacja).
    3. Utwórz obiekty ImageDataset dla każdego z wymienionych zbiorów danych:
       - Zbiór treningowy oznaczony (labeled).
       - Zbiór treningowy nieoznaczony (unlabeled).
       - Zbiór testowy.
    4. Zainicjalizuj DataLoadery dla każdego zbioru, ustawiając odpowiednie parametry, takie jak:
       - `batch_size`
       - `shuffle` (dla zbioru treningowego).
       - `num_workers` (dla optymalizacji wydajności).
    5. Upewnij się, że DataLoadery zwracają dane w odpowiednim formacie (np. (image, label) dla labeled, tylko image dla unlabeled).
    6. Przetestuj poprawność działania DataLoaderów na przykładzie małego batcha danych.
    
    Zwróć:
    - `train_labeled_loader`: DataLoader dla oznaczonego zbioru treningowego.
    - `train_unlabeled_loader`: DataLoader dla nieoznaczonego zbioru treningowego.
    - `test_loader`: DataLoader dla zbioru testowego.
    """
    return train_labeled_loader, train_unlabeled_loader, test_loader


In [ ]:
class YourModel(nn.Module):
    def __init__(self):
        super(YourModel, self).__init__()
        """
        TODO: Zdefiniuj architekturę swojego modelu.
        - W sekcji self-supervised możesz skupić się na encoderze (np. CNN).
        - W sekcji fine-tuning dodaj warstwę klasyfikacyjną.
        """
        pass

    def forward(self, x):
        """
        TODO: Zaimplementuj przepływ danych przez model.
        - Zwróć wynik forward pass.
        """
        pass

In [ ]:
def self_supervised_pretraining(unlabeled_loader, model, epochs, lr):
    """
    Pretraining modelu w trybie self-supervised.

    Args:
        unlabeled_loader (DataLoader): Dane bez etykiet.
        model (nn.Module): Model do pretrenowania.
        epochs (int): Liczba epok pretrenowania.
        lr (float): Współczynnik uczenia.

    TODO:
    1. Zaimplementuj metodę self-supervised:
       - Możesz użyć autoenkodera, kontrastowego uczenia (SimCLR, BYOL), itp.
    2. Ustaw odpowiednią funkcję straty i optymalizator.
    3. Przeprowadź trening i monitoruj utratę (loss).
    """
    pass


In [ ]:
def fine_tune_model(labeled_loader, model, epochs, lr):
    """
    Fine-tuning modelu na danych z etykietami.

    Args:
        labeled_loader (DataLoader): Dane z etykietami.
        model (nn.Module): Model do trenowania.
        epochs (int): Liczba epok treningu.
        lr (float): Współczynnik uczenia.

    TODO:
    1. Dodaj warstwę klasyfikacyjną do modelu.
    2. Ustaw funkcję straty (np. CrossEntropyLoss) i optymalizator.
    3. Wytrenuj model na zbiorze labeled.
    """
    pass

In [ ]:

def save_predictions(test_loader, model, output_file="odpowiedzi.csv"):
    """
    Generuje predykcje na danych testowych i zapisuje je do pliku CSV.
    
    Args:
        test_loader (DataLoader): Dane testowe bez etykiet.
        model (nn.Module): Wytrenowany model.
        output_file (str): Nazwa pliku wyjściowego (domyślnie 'predictions.csv').
    
    TODO:
    1. Wykonaj predykcje dla wszystkich obrazów w test_loader.
    2. Zapisz ścieżki do obrazów oraz przewidywane etykiety w formacie CSV.
       - Format CSV: 
         | image_path | label |
    """
    # Model w trybie ewaluacyjnym
    model.eval()
    predictions = []

    with torch.no_grad():
        for images in test_loader:
            # TODO: Wykonaj predykcje (np. torch.max dla klasyfikacji).
            # Pamiętaj, aby zebrać ścieżki do obrazów i przewidywane etykiety.
            pass

    # TODO: Zapisz wynik do pliku CSV w odpowiednim formacie.
    pass


In [ ]:
def main():
    # TODO: Załaduj dane
    labeled_loader, unlabeled_loader, test_loader = load_datasets()

    # TODO: Stwórz model
    # model = YourModel()

    # TODO: Pretraining
    # print("Rozpoczęcie pretrenowania...")
    # self_supervised_pretraining(unlabeled_loader, model, epochs=..., lr=...)

    # TODO: Fine-tuning
    # print("Rozpoczęcie fine-tuningu...")
    # fine_tune_model(labeled_loader, model, epochs=..., lr=...)

    # TODO: Generacja predykcji na zbiorze testowym
    # print("Generowanie predykcji na danych testowych...")
    # save_predictions(test_loader, model, output_file="odpowiedzi.csv")


if __name__ == "__main__":
    main()